## Data Cleaning For Group Question: Runtime & Profitability
##### Steven Zych
This notebook takes two .csv files, joins them, and cleans the resulting table. This final table contains information on films, their runtimes, their profitability, and their release dates.

---

Importing data and two main dataframes:

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# This pulls all the .csv files into one list of dataframes.

csv_files = glob('zippedData/*.csv.gz')
csv_dfs = [pd.read_csv(f) for f in csv_files]
len(csv_dfs)

9

In [3]:
csv_dfs[3].head() # Runtimes

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
csv_dfs[8].head() # Domestic gross

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


Joining our two dataframes into one:

In [5]:
# Join on movie title. Inner so no near-duplicates with spelling inconsistencies.

joined = csv_dfs[8].join(csv_dfs[3].set_index('primary_title'), on=['movie'], how='inner')
print(len(joined))
joined.head()

3815


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,tconst,original_title,start_year,runtime_minutes,genres
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",tt1775309,Abatâ,2011,93.0,Horror
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,136.0,"Action,Adventure,Fantasy"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",tt6565702,Dark Phoenix,2019,113.0,"Action,Adventure,Sci-Fi"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",tt2395427,Avengers: Age of Ultron,2015,141.0,"Action,Adventure,Sci-Fi"
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",tt4154756,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi"


Checking for fully-duplicated rows and missing data throughout:

In [6]:
# No full duplicates!
joined.duplicated().sum()

0

In [7]:
# Some missing data on runtime
joined.isna().sum()

id                     0
release_date           0
movie                  0
production_budget      0
domestic_gross         0
worldwide_gross        0
tconst                 0
original_title         1
start_year             0
runtime_minutes      487
genres                72
dtype: int64

Dropping the missing rows, columns we don't need, and other repeated entries:

In [8]:
# Drop those missing runtime rows
nulldrop = joined.dropna(axis=0)
nulldrop.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,tconst,original_title,start_year,runtime_minutes,genres
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",tt1775309,Abatâ,2011,93.0,Horror
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,136.0,"Action,Adventure,Fantasy"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",tt6565702,Dark Phoenix,2019,113.0,"Action,Adventure,Sci-Fi"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",tt2395427,Avengers: Age of Ultron,2015,141.0,"Action,Adventure,Sci-Fi"
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",tt4154756,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi"


In [9]:
# Drop columns we don't need. Set new index
columndrop = nulldrop.set_index('tconst').drop(['id', 'original_title', 'release_date'], axis=1)
columndrop.head()

,movie,production_budget,domestic_gross,worldwide_gross,start_year,runtime_minutes,genres
tconst,,,,,,,
tt1775309,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2011,93.0,Horror
tt1298650,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy"
tt6565702,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,113.0,"Action,Adventure,Sci-Fi"
tt2395427,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,141.0,"Action,Adventure,Sci-Fi"
tt4154756,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018,149.0,"Action,Adventure,Sci-Fi"


In [10]:
# Drop duplicates based on repeat names.
# Using release date too incase of remake/franchise.

# I made a Pandas series of booleans, reversed that series with a new
# one, then passed that series to the dataframe.

x = (columndrop.movie.duplicated() & columndrop.start_year.duplicated())
y = []
for value in x:
    if value == True:
        y.append(False)
    else:
        y.append(True)
        
nodupes = columndrop[y]
nodupes.head()

,movie,production_budget,domestic_gross,worldwide_gross,start_year,runtime_minutes,genres
tconst,,,,,,,
tt1775309,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2011,93.0,Horror
tt1298650,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy"
tt6565702,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,113.0,"Action,Adventure,Sci-Fi"
tt2395427,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,141.0,"Action,Adventure,Sci-Fi"
tt4154756,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018,149.0,"Action,Adventure,Sci-Fi"


In [11]:
nodupes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2203 entries, tt1775309 to tt2107644
Data columns (total 7 columns):
movie                2203 non-null object
production_budget    2203 non-null object
domestic_gross       2203 non-null object
worldwide_gross      2203 non-null object
start_year           2203 non-null int64
runtime_minutes      2203 non-null float64
genres               2203 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 137.7+ KB


Profit and runtime .csv file ready to export:

In [12]:
runtime_profit = nodupes

In [13]:
runtime_profit.to_csv(r'zippedData/runtime_profit.csv')